# A deeper look at SVM architectures

In the last experiment that we ran, where we used a LinearSVM without changing any of the deafult parameters or feaure engineering, we achieved an accuracy of 63%. It also gave us convergence warnings for the linear kernel.
In this notebook we will iterate over the SVM design and try different approaches to the problem using this classifier.
Let's import what we need.

In [13]:
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from exp4_data_feature_extraction import get_balanced_dataset
from scripts import training_helpers

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import NuSVC

In [15]:
reviews_set, fake_reviews, genuine_reviews, unused_genuine_reviews = get_balanced_dataset()

In [82]:
all_reviews = reviews_set[:40000]

X = [x.review_content for x in all_reviews]
y = [1 if x.label else 0 for x in all_reviews]

## Start with Logistic Regression


~~## No AUROC or F1 Score with LinearSVC~~

~~Since LinearSVC uses LibLinear it is not possible to retrieve the probabilities to calculate the AUROC or F1 score. The SVC implementations in sklearn have an option to return probabilities which can be used to calculate the AUC, however Liblinear does not provide this. It is possible to use CalibratedClassifierCV to obtain probabilities, however this also tries to tune the hyperparameters and severely limits the amount of data we can ue. If we want to use LinearSVC we will have to leave this out for now. It is also possible to use SVC with a linear kernel, but the LinearSVM is known to perform better.~~

~~Instead of this, I will use Logistic Regression which is very similar to LinearSVC, and performs similarly. Logistic regression will allow me to view these metrics and once I have explored how to improve this I can switch back to LinearSVC. Both classifiers attempt to divide the samples, and logistic regression is known to do a good job at producing a wide margin in it's division, which is what SVMs try to do.~~

In [17]:
from sklearn.metrics import roc_auc_score

def auroc_score_from_probabilities(probabilities, labels):
  true_probabilities = [probabilities[i][1] for i in range(0, len(labels))]
  return roc_auc_score(labels, true_probabilities)

In [18]:
import unittest

class TestRocScoreFromProbabilities(unittest.TestCase):
  
  def test_uses_correct_probabilities(self):
    probabilities = [[0.9, 0.1], [0.1, 0.9]]
    labels = [0, 1]
    self.assertEquals(1, auroc_score_from_probabilities(probabilities, labels))

unittest.main(argv=[''], verbosity=2, exit=False)

test_uses_correct_probabilities (__main__.TestRocScoreFromProbabilities) ... /home/stefan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Please use assertEqual instead.
  
ok

----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


We define a function to automate the process of cross validation and finding the accuracy, the mean and the variance:

In [19]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, log_loss
import numpy as np
def run_cross_validate(model, X, y, get_features_fn, cv=5):
  skfSplitter = StratifiedKFold(n_splits=cv)
  metrics = {
    "accuracies": [],
    "auroc": [],
    "f1 scores": [],
    "log loss": []
  }
    
  false_negatives = 0
  false_positives = 0
  for train_indices, test_indices in skfSplitter.split(X, y):
    training_X = get_features_fn([X[x] for x in train_indices], fit=True)
    training_y = [y[x] for x in train_indices]
    test_X = get_features_fn([X[x] for x in test_indices])
    test_y = [y[x] for x in test_indices]
  
    model.fit(training_X, training_y)
    probabilities = model.predict_proba(test_X)
    
    predicted = [0 if x[0] > x[1] else 1 for x in probabilities]
    for i in range(0, len(predicted)):
      if (predicted[i] == 0 and test_y[i] == 1):
        false_negatives+=1
      if (predicted[i] == 1 and test_y[i] == 0):
        false_positives+=1
    
    metrics["accuracies"].append(accuracy_score(test_y, predicted))
    metrics["auroc"].append(auroc_score_from_probabilities(probabilities, test_y))
    metrics["f1 scores"].append(f1_score(test_y, predicted))
    metrics["log loss"].append(log_loss(test_y, probabilities))

  num_samples = len(X)
  metrics["mean accuracy"] = np.mean(metrics["accuracies"])
  metrics["mean variance"] = np.var(metrics["accuracies"])
  metrics["mean auroc"] = np.mean(metrics["auroc"])
  metrics["mean f1 scores"] = np.mean(metrics["f1 scores"])
  metrics["mean log loss"] = np.mean(metrics["log loss"])
  metrics["false negatives rate"] = (false_negatives / cv) / num_samples
  metrics["false positives rate"] = (false_positives / cv) / num_samples
  return metrics

In [20]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver="liblinear")

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
cVec = CountVectorizer()
def get_features(predictor_set, fit=False):
  if fit:
    return cVec.fit_transform(predictor_set)
  return cVec.transform(predictor_set)
run_cross_validate(classifier, X, y, get_features)

{'accuracies': [0.6290927268182954,
  0.6260934766308422,
  0.622,
  0.6186546636659165,
  0.6314078519629908],
 'auroc': [0.674600336732139,
  0.662692647715114,
  0.6595396089641201,
  0.6634598501148327,
  0.6728436811060747],
 'f1 scores': [0.6490066225165562,
  0.6407300672430355,
  0.637410071942446,
  0.6356033452807646,
  0.6473684210526315],
 'log loss': [0.8076657932390442,
  0.833224635660126,
  0.8415995707594071,
  0.8324635908498775,
  0.8138489881298778],
 'mean accuracy': 0.6254497438156089,
 'mean variance': 2.145172328504902e-05,
 'mean auroc': 0.6666272249264562,
 'mean f1 scores': 0.6420237056070868,
 'mean log loss': 0.8257605157276666,
 'false negatives rate': 0.03295,
 'false positives rate': 0.041960000000000004}

## Metrics to evaluate

### Accuracy
This is the percentage of correct classifications. The higher the better, however it is not appropriate in all cases.

This metric falls to the imbalanced classification problem. When there are many more of one class, the classifier can choose it much more, or all the time, to achieve a high accuracy.

### False Positives, False Negatives
False positives are samples that were classified as fake, but were in fact genuine. False negatives are samples that were classified as genuine, but were in fact fake. We want to reduce both of these values as much as possible. True positives and true negatives are the adverse values which we want to maximise, although they correlate to false positives and negatives so there's no need to include them here.

One important question here is **False negatives vs False positives**? Is it worse to falsely suggest that something is fake, or is it worse to falsely suggest that something is genuine? Probably in this system a human might be paid to read those suspicious reviews. It would be good to catch all the fake reviews, plus some genuine ones, because this is just like filtering the huge number of reviews to make a human's job easier. In this case it is better to reduce false negatives. If humans are not reviewing the system, then this would be a different situation, it would probably be better to reduce false positives.

### Recall
Of all the fake reviews, what percentage were identified as fake? This is not subject to the imbalanced
classification problem. We aim to maximise it as an indication of how well we are really identifying our fake reviews. 

We cannot focus solely on recall, because we could identify all reviews as fake and achieve 100% recall. Precision must be included in the consideration.

### Precision
Of all the reviews identified as fake, what percentage are actually fake? If we classify all reviews as fake, then our precision will be low. If we classify all reviews as genuine, then we wont have any precision either.

In our case it might be more important to have a high recall, if we don't want to miss any fake reviews. Otherwise if we want to be as accurate as possible we can balance recall and precision.

### F1 Score
This is a harmonic mean of precision and recall. Because of this it punishes extreme values such as a recall of or a precision of 0.0

This also acts as a single number metric representing precision and recall.

### Area Under Curve (AUC) 
This gives us a measure of discrimination, how well we correctly classify both classes. This does not use a 'Yes' or 'No' which can make it more interesting than accuracy.

At different classification thresholds, how well do we predict fake reviews as 'more fake' than genuine reviews. We plot the true positive rate against the false positive rate to get a graph. Changing the threshold allows us to create a graph because at low thresholds we will have more fake reviews, increasing the true positives rate. Decreasing the treshold means we will have less genuine reviews, decreasing the true negative rate, which therefore increases the false positive rate.

An AUC of 0.8 means the chance of the model distinguishing positive and negative classes is 80%.

### Mean Squared Error
The average of the square difference between the original values and the predicted values. Adds focus to large errors, and is easier to compute than mean absolute error.

The closer the mean squared error is to zero the better. It incorporates the variance and the bias of the estimator

### Logarithmic loss
This takes into account the uncertainty of a prediction, taking into account how far away from the actual label the prediction is. As the probability approaches correct the log loss reduces only very little. As the probability approaches incorrect the log loss increases rapidly. This means that confident incorrect values are highly penalized.

We aim to minimize log loss.

### Cohen's Kappa
A reliability metric used when there is more than one classifier. Computes an agreement percentage of the used classifiers. It is out of scope at this stage.

## Current Results

Our results show quite normal results. The AUROC score and F1 score above are as expected, reflective of similar work in this field. We can see that the variance is not very large, so we can depend on our scores.

Since there is nothing alarming, the next thing to consider is improving features or trying to find more predictive features. We might see better results if convert our bow to tfidf. ~~We might see better results if we preprocess our text to lemmatize and remove stopwords. Since bag of words is our main feature here, this should hopefully be influential. In this case we are removing all of the stopwords, which may be a bad idea. We can't know for sure unless we experiment.~~

In [57]:
def tf_idf_bag_of_words(cv, tfidf, predictor_set, fit):
  if fit:
    return tfidf.fit_transform(cv.fit_transform(predictor_set))
  else:
    return tfidf.transform(cv.transform(predictor_set))

In [22]:
from sklearn.feature_extraction.text import TfidfTransformer

count_vectorizer = CountVectorizer()
tfidf_transformer = TfidfTransformer()
def get_features(predictor_features, fit=False):
  return tf_idf_bag_of_words(count_vectorizer, tfidf_transformer, predictor_set, fit)

run_cross_validate(classifier, X, y, get_features)

{'accuracies': [0.6523369157710572,
  0.6570857285678581,
  0.66425,
  0.6614153538384596,
  0.6631657914478619],
 'auroc': [0.7170031989053457,
  0.716968341281871,
  0.7200749951687392,
  0.7161055019092074,
  0.7171423969120514],
 'f1 scores': [0.6574735286875153,
  0.6568284142071036,
  0.6669972725018597,
  0.6621756487025948,
  0.6618127039919659],
 'log loss': [0.615738420115528,
  0.6172708805448116,
  0.6148862153132552,
  0.6180723830800674,
  0.6177006818375328],
 'mean accuracy': 0.6596507579250475,
 'mean variance': 1.933878994497638e-05,
 'mean auroc': 0.717458886835443,
 'mean f1 scores': 0.6610575136182077,
 'mean log loss': 0.616733716178239,
 'false negatives rate': 0.03375,
 'false positives rate': 0.034319999999999996}

Looks like switching to Tfidf had a positive effect on our results, which is not unexpected. Let's have a look at our vocabulary, to see what words are actually in our bag of words. Let's try to clean our features more, hopefully a smaller number of features, more concentrated with quality words will improve accuracy. Currently our bag of words has the following shape: ~~Let's be adventurous and try using a bag of words that only includes stopwords, on the hypothesis that it is the way people say things that is predictive of a fake review, rather than what they talk about.~~ 

In [23]:
count_vectorizer = CountVectorizer()
count_vectorizer.fit_transform(X).shape

(20000, 33653)

Let's set a low limit for the min document frequency. This cleans out everything from the typos to the random jibberish:

In [24]:
count_vectorizer = CountVectorizer(min_df=4)
count_vectorizer.fit_transform(X).shape

(20000, 11533)

In [25]:
run_cross_validate(classifier, X, y, get_features)

{'accuracies': [0.6518370407398151,
  0.6550862284428893,
  0.66525,
  0.6559139784946236,
  0.6636659164791198],
 'auroc': [0.71634402750229,
  0.7159391043564783,
  0.7199524948931135,
  0.7156260114447396,
  0.7168274889857265],
 'f1 scores': [0.6573185731857318,
  0.6563745019920318,
  0.6681536555142503,
  0.6575410652065704,
  0.6624843161856964],
 'log loss': [0.6160163377452522,
  0.6176232360162895,
  0.6149526840588501,
  0.6183856677609213,
  0.6180224249695648],
 'mean accuracy': 0.6583506328312895,
 'mean variance': 2.6974801977417865e-05,
 'mean auroc': 0.7169378254364696,
 'mean f1 scores': 0.6603744224168562,
 'mean log loss': 0.6170000701101755,
 'false negatives rate': 0.0337,
 'false positives rate': 0.03463}

Now let's try adding reviewer features, which have been proven to be predictive:

In [59]:
from sklearn.preprocessing import StandardScaler

def scaled_reviewer_features(reviews):
  reviewer_reviews = reviews_by_reviewer(reviews)
  reviewer_predictors = [list(reviewer_features(x.user_id, reviewer_reviews)) for x in reviews]
  return StandardScaler().fit_transform(reviewer_predictors)

In [60]:
from exp2_feature_extraction import reviewer_features, reviews_by_reviewer
from scipy.sparse import coo_matrix, hstack

count_vectorizer = CountVectorizer(max_features=1000, min_df=4)
tfidf_transformer = TfidfTransformer()
def get_features(predictor_features, fit=False):
  predictor_features_text = [x.review_content for x in predictor_features]
  bow = tf_idf_bag_of_words(count_vectorizer, tfidf_transformer, predictor_features_text, fit)
  reviewer_scaled = scaled_reviewer_features(predictor_features)
  return hstack([coo_matrix(reviewer_scaled), bow])

run_cross_validate(LogisticRegression(solver="liblinear"), all_reviews, y, get_features)

{'accuracies': [0.6563359160209947,
  0.6598350412396901,
  0.6695,
  0.6666666666666666,
  0.6649162290572643],
 'auroc': [0.7182167189655957,
  0.71681791626874,
  0.7240008790019778,
  0.7220881271957886,
  0.7216891771223268],
 'f1 scores': [0.659400544959128,
  0.6576100628930817,
  0.6734189723320159,
  0.6681603186457555,
  0.6629778672032193],
 'mean accuracy': 0.663450770596923,
 'mean variance': 2.2555477364558655e-05,
 'mean auroc': 0.7205625637108858,
 'mean f1 scores': 0.6643135532066401}

Let's try adding some more features, this time for review details like date posted, the id of the reviewer and the id of the product:

In [85]:
count_vectorizer = CountVectorizer(max_features=1000, min_df=4)
tfidf_transformer = TfidfTransformer()

from datetime import datetime as dt
def extract_date_ordinals(reviews):
  return [dt.strptime(x.date, '%Y-%m-%d').date().toordinal() for x in reviews]

def get_features(predictor_features, fit=False):
  predictor_features_text = [x.review_content for x in predictor_features]
  bow = tf_idf_bag_of_words(count_vectorizer, tfidf_transformer, predictor_features_text, fit)
  reviewer_scaled = scaled_reviewer_features(predictor_features)

  date_ordinals = extract_date_ordinals(predictor_features)
  review_details = StandardScaler().fit_transform([[date_ordinals[i], predictor_features[i].user_id, predictor_features[i].product_id] for i in range(len(predictor_features))])
    
  return hstack([coo_matrix(reviewer_scaled), coo_matrix(review_details), bow])

run_cross_validate(LogisticRegression(solver="liblinear"), all_reviews, y, get_features)

{'accuracies': [0.7041619797525309,
  0.701375,
  0.716875,
  0.706625,
  0.7078384798099763],
 'auroc': [0.7768046069689891,
  0.7701108361861083,
  0.7833027779703136,
  0.7760386161954945,
  0.7773563997037246],
 'f1 scores': [0.7068004459308808,
  0.7032666749472115,
  0.718038092866924,
  0.7066616672915886,
  0.7095090118085768],
 'mean accuracy': 0.7073750919125014,
 'mean variance': 2.747016252053271e-05,
 'mean auroc': 0.7767226474049259,
 'mean f1 scores': 0.7088551785690363}

Looks like this gave us a boost.

Although reviewer features are predictive alone, here they are doing very little to increase the accuracy. Let's ~~forget about them and~~ try switching to LinearSVC now. We have to drop some of our metrics based on probabilities because the underlying implementation of LinearSVC does not expose them to us:

In [49]:
def run_cross_validate(model, X, y, get_features_fn, cv=5):
  skfSplitter = StratifiedKFold(n_splits=cv)
  metrics = {
    "accuracies": [],
    "auroc": [],
    "f1 scores": [],
  }
    
  for train_indices, test_indices in skfSplitter.split(X, y):
    training_X = get_features_fn([X[x] for x in train_indices], fit=True)
    training_y = [y[x] for x in train_indices]
    test_X = get_features_fn([X[x] for x in test_indices])
    test_y = [y[x] for x in test_indices]
  
    model.fit(training_X, training_y)
    scores = model.decision_function(test_X)
    
    predicted = [1 if score >= 0 else 0 for score in scores]
    metrics["accuracies"].append(accuracy_score(test_y, predicted))
    metrics["auroc"].append(roc_auc_score(test_y, scores))
    metrics["f1 scores"].append(f1_score(test_y, predicted))

  num_samples = len(X)
  metrics["mean accuracy"] = np.mean(metrics["accuracies"])
  metrics["mean variance"] = np.var(metrics["accuracies"])
  metrics["mean auroc"] = np.mean(metrics["auroc"])
  metrics["mean f1 scores"] = np.mean(metrics["f1 scores"])
  return metrics

In [66]:
run_cross_validate(LinearSVC(max_iter=10000), all_reviews, y, get_features)

{'accuracies': [0.6523369157710572,
  0.6513371657085728,
  0.6665,
  0.6559139784946236,
  0.6559139784946236],
 'auroc': [0.7106614852496032,
  0.7062664261865461,
  0.7154473597565596,
  0.7129525457016693,
  0.7154735599903351],
 'f1 scores': [0.656458384786367,
  0.6490566037735849,
  0.6696384348687469,
  0.6594059405940595,
  0.6542713567839196],
 'mean accuracy': 0.6564004076937755,
 'mean variance': 2.8924675498047935e-05,
 'mean auroc': 0.7121602753769427,
 'mean f1 scores': 0.6577661441613355}

It looks like the accuracy is pretty similar, as expected. Now let's try changing to a sigmoid kernel:

In [71]:
from sklearn.svm import NuSVC
run_cross_validate(NuSVC(kernel="sigmoid", gamma="scale"), all_reviews, y, get_features)

{'accuracies': [0.4988752811797051,
  0.5023744063984004,
  0.51225,
  0.4988747186796699,
  0.5001250312578145],
 'auroc': [0.6055893637021945,
  0.6228025329373307,
  0.6281006632264923,
  0.6247840792157267,
  0.6178083308779352],
 'f1 scores': [0.6502703645560789,
  0.6531963072635428,
  0.663910422049957,
  0.6522040958000694,
  0.6554042406481642],
 'mean accuracy': 0.502499887503118,
 'mean variance': 2.5400000278715447e-05,
 'mean auroc': 0.6198169939919358,
 'mean f1 scores': 0.6549970860635626}

In [ ]:
import string

from exp2_feature_extraction import find_words
x_words = [find_words(x) for x in X]

count_vectorizer.fit_transform([" ".join([y for y in x if y not in string.punctuation]) for x in x_words]).shape

In [ ]:
from sklearn.preprocessing import StandardScaler

count_vectorizer.set_params(vocabulary=None, stop_words=None)

count_vectorizer = CountVectorizer()
model = Pipeline([
  ('cv', count_vectorizer),
  ('tfidf', TfidfTransformer()),
  ('classifier', LogisticRegression(solver="liblinear"))
])
run_cross_validate(model, X, y)

In [ ]:
from exp2_feature_extraction import find_words, preprocess_words
def preprocess(review_content): # Not adding bigrams yet
  return " ".join(preprocess_words(find_words(review_content), bigrams=False))

In [ ]:
review_content = all_reviews[0].review_content
print(review_content)
preprocess(review_content)

In [ ]:
X_lemmatized = [preprocess(x.review_content) for x in all_reviews]

Results for lemmatized words with all stopwords and all words with <= 3 characters removed:

In [ ]:
run_cross_validate(model, X_lemmatized, y)

This shows only a slightly better result. Perhaps the different versions of words people use are actually important, and perhaps stopwords are important here too. At least more important than other tasks, for example identifying sentiment or topic. Let's try lemmatizing, but without removing stopwords.

In [ ]:
def preprocess(review_content): # Not adding bigrams yet
  return " ".join(preprocess_words(find_words(review_content), bigrams=False, stopwords=[]))

In [ ]:
review_content = all_reviews[0].review_content
print(review_content)
preprocess(review_content)

In [ ]:
training_helpers.get_accuracy(model, [preprocess(x.review_content) for x in all_reviews], y, 5)